## Link to article

This notebook is included in the documentation, where the interactive Plotly charts show up. See:
https://pegasystems.github.io/cdh-datascientist-tools/Python/articles/vf_analysis.html

In [ ]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import plotly.io as pio
pio.renderers.default='notebook_connected'

import sys
sys.path.append('../../../')

# Value Finder analysis
Pega value finder simulations create a dataset called 'pyValueFinder'. This dataset contains the decision records including the stage labels that is used for the analysis that results in the typical Value Finder pie chart.

In the data folder we've stored a copy of such as dataset, generated from an (internal) demo application (CDHSample). 

From CDH Tools 2.2, we've added a new ValueFinder class, which allows you to easily analyze these datasets.

This page shows an example how the data can be used for additional analyses.

First, let's look at the results as presented in Pega:

![Pega value finder screen](pegarun_8_6_0.png)

For the sample provided, the under-served threshold is 1.2%. There are 10.000 customers, 3491 without actions, 555 under-served and 5954 well-served.

Now, let's import our class, read the data and try to recreate this view. Just like with the ADMDatamart class, you can supply your own path and filename as such:
```python
ValueFinder(path = 'path-to-data', filename="Data-Insights_pyValueFinder_timestamp_GMT.zip")
```

If only a path is supplied, it will automatically look for the latest file. 
It is also possible to supply a dataframe as the 'df' argument directly, in which case it will use that instead. 
Lastly, there is now also an additional dataset in cdh tools, which is what we'll be using.

In [ ]:
from cdhtools import ValueFinder, datasets
import polars as pl
vf = datasets.SampleValueFinder()

As we can see, it has found a file on the GitHub repo and imports it straight from there. It also prints out some extra information about some calculations, which can be suppressed by supplying the keyword 'verbose=False'. 

Since there is only one dataset, the data is simply stored in the attribute 'df'. We heavily filter out the dataset for performance reasons, so the data will look like this:

In [ ]:
vf.df.head(5)

This is already enough information to generate the same piechart as shown in platform, but to replicate the same values, we would need to compute the propensity threshold. In this case, the quantile of `0.052` of the propensity distribution seems to represent the same counts as in platform. With that, simply call the `plotPieCharts()` function on the data:

In [ ]:
vf.plotPieCharts(0.052, verbose=False)

By hovering over the rightmost pie chart, you can see the numbers match up exactly to that shown in platform. What's more, we don't just show the counts in the final arbitration stage, but also the distribution of the eligibility, applicability, and the suitability stages of Next-Best-Action Designer. This gives some idea on how aggressively you may be filtering out actions between different stages. 

Now, of course, if a customer is well served or not, depends heavily on what we consider to be well served. For this, we set a threshold based on the 5th percentile, through the distribution in the first stage. We can plot what that will look like as such, where the dotted line is that set threshold:

In [ ]:
vf.plotPropensityThreshold()

We can also look at the propensity distributions across the different stages. Simply call `plotPropensityDistribution()`.

In [ ]:
vf.plotPropensityDistribution()

So there is a spread out distribution of propensities - making a proper threshold a relevant topic. While we can compute this pie chart for one threshold, we can also do this for a range of them. To do this, simply supply three arguments to the `plotPieCharts()` function: `start`, `stop` and `step`. These correspond to a range of propensity *quantiles* for which we want to compute the counts. In the background, this will generate the aggregated counts per stage, which we can plot as such:

In [ ]:
vf.plotPieCharts(start=0.01, stop=0.5, step=0.01, verbose=False)

Note the slider at the bottom: playing around with this, you can easily see how choosing a different threshold changes the view of a customer. This makes intuitive sense: if you consider an offer 'good' from a lower propensity threshold, then more customers will be well served than if you consider an offer 'good' from a higher propensity threshold.

While this is a nice 'slice' of the distribution at a given threshold, we can also show a bit more information. Call `plotPropensityDistributionPerThreshold()` to show this same distribution, but then with the threshold on the x-axis. By default, it considers the quantiles, but if you supply the `target` parameter to be `'Propensity'`, then it will update to that instead.

In [ ]:
vf.plotDistributionPerThreshold(verbose=False)

In [ ]:
vf.plotDistributionPerThreshold(target='Propensity', verbose=False)

Lastly, it may also be interesting to instead look at the actions themselves, throughout the stages. Simply call the `plotFunnelChart()` function for an overview of this funnel effect throughout each stage. As a rule of thumb, if there are only a few actions in each stage, this is not a good sign. If certain actions are completely filtered out from one stage to the next, it may also be a warning of agressive filtering. In this case, let's also use the 'query' functionality to only look at actions in the `'Sales'` issue.

In [ ]:
vf.plotFunnelChart('Action', query=pl.col('pyIssue')=='Sales')

Of course this is quite a lot of information. If instead we want to look at the distribution of *issues* over each stage, simply supply the `level` parameters as `'Issue'`:

In [ ]:
vf.plotFunnelChart('Issue')

Lastly, it may also be interesting to look at the distribution of groups over the different stages. Here, let's again filter on the `'Sales'` issue only.

In [ ]:
vf.plotFunnelChart('Group', query=pl.col('pyIssue')=='Sales')